## **Load Libraries**

In [1]:
import torch
import copy
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets, transforms, models
from tqdm import tqdm

from sklearn.metrics import confusion_matrix, classification_report

import warnings
warnings.filterwarnings('ignore')

## **Parameter Initialization**

In [2]:
batch_size = 16
learning_rate = 0.001
epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## **Data Loading and Preparation**

In [3]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

def load_dataset():
  train_data = datasets.CIFAR10(root='/content/data', train=True, download=True, transform=transform)
  train_size = int(0.8 * len(train_data))
  val_size = len(train_data) - train_size
  train_data, val_data = random_split(train_data, [train_size, val_size])
  test_data = datasets.CIFAR10(root='/content/data', train=False, download=True, transform=transforms.ToTensor())

  train_subset_indices, test_subset_indices = list(range(batch_size*4 * 24)), list(range(batch_size*4 * 8))
  train_subset = Subset(train_data, train_subset_indices)
  val_data = Subset(val_data, test_subset_indices)
  test_data = Subset(test_data, test_subset_indices)

  return train_subset, val_data, test_data

def prepare_data(train_data, val_data, test_data=None):
  train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
  if test_data:
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

  return train_loader, val_loader



## **Model Loading**

In [4]:
def load_model(pretrained=True, width_mult=1.0):
  model = models.mobilenet_v2(pretrained=pretrained, width_mult=width_mult)
  model.classifier[1] = nn.Linear(model.last_channel, 10)
  return model.to(device)


## **Evaluation**

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_labels.extend(labels.numpy())
            all_predictions.extend(predicted.cpu().numpy())

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_predictions)
    print("Confusion Matrix:")
    print(cm)

    # Classification report (Precision, Recall, F1-score)
    report = classification_report(all_labels, all_predictions)
    print("Classification Report:")
    print(report)

## **Fine-tuning**

In [6]:
def train_model(model, train_loader, val_loader, epochs, gradient_acc_steps = 256):
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  best_model = copy.deepcopy(model).to(device)
  best_accuracy = 0.0

  for epoch in range(epochs):
    print(f"*** Epoch: {epoch+1} ***")
    model.train()
    running_loss = 0.0
    n_steps = 0
    bar = tqdm(train_loader)
    for images, labels in bar:

      images, labels = images.to(device), labels.to(device)
      outputs = model(images)

      unit_loss = criterion(outputs, labels)*len(images)/gradient_acc_steps
      unit_loss.backward()
      running_loss += unit_loss.detach().item()

      n_steps += len(images)
      if n_steps % gradient_acc_steps == 0:
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        bar.set_description(f"Loss: {running_loss:.5f}")

        optimizer.zero_grad()
        running_loss = 0.0

    model.eval()
    with torch.no_grad():
      correct = 0
      total = 0
      for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

      val_accuracy = 100 * correct / total

      print(f"Validation Accuracy: {val_accuracy:.2f}%")

      if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model = copy.deepcopy(model).to(device)

  return best_model

## **Model Distillation**

In [7]:
def distillation_loss(teacher_outputs, student_outputs, labels, alpha, temperature):
  teacher_probs = nn.functional.softmax(teacher_outputs / temperature, dim=1)
  student_probs = nn.functional.log_softmax(student_outputs / temperature, dim=1)
  kd_loss = nn.functional.kl_div(student_probs, teacher_probs, reduction='batchmean')*(temperature**2)
  ce_loss = nn.functional.cross_entropy(student_outputs, labels)
  return alpha * kd_loss + (1 - alpha) * ce_loss


def model_distillation(teacher_model, student_model, train_loader, val_loader,
                    epochs, alpha=0.5, temperature=3, gradient_acc_steps=256):

  optimizer = torch.optim.Adam(student_model.parameters(), lr=learning_rate)


  best_model = copy.deepcopy(student_model).to(device)
  best_accuracy = 0.0

  teacher_model.eval()

  for epoch in range(epochs):
    print(f"*** Epoch: {epoch+1} ***")
    student_model.train()
    running_loss = 0.0

    n_steps = 0
    bar = tqdm(train_loader)
    for images, labels in bar:
      images, labels = images.to(device), labels.to(device)
      with torch.no_grad():
        teacher_outputs = teacher_model(images)
      student_outputs = student_model(images)

      unit_loss = distillation_loss(
          teacher_outputs, student_outputs, labels, alpha, temperature
      )*len(images)/gradient_acc_steps
      unit_loss.backward()
      running_loss += unit_loss.detach().item()

      n_steps += len(images)
      if n_steps % gradient_acc_steps == 0:
        nn.utils.clip_grad_norm_(student_model.parameters(), 1.0)
        optimizer.step()

        bar.set_description(f"Loss: {running_loss:.5f}")

        optimizer.zero_grad()
        running_loss = 0.0

    student_model.eval()
    with torch.no_grad():
      teacher_correct = 0
      student_correct = 0
      total = 0
      for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        teacher_outputs = teacher_model(images)
        student_outputs = student_model(images)
        _, teacher_predicted = torch.max(teacher_outputs.data, 1)
        _, student_predicted = torch.max(student_outputs.data, 1)
        total += labels.size(0)
        teacher_correct += (teacher_predicted == labels).sum().item()
        student_correct += (student_predicted == labels).sum().item()


      val_accuracy = 100 * student_correct / total

      if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model = copy.deepcopy(student_model).to(device)

      print(f"Validation Accuracy:- Teacher: {100*teacher_correct/total:.2f}%; Student: {val_accuracy:.2f}%")

  return best_model

## **main**

In [8]:
if __name__ == '__main__':
  train_data, val_data, test_data = load_dataset()
  print("Train, validation, and test data size:")
  print(len(train_data), len(val_data), len(test_data))
  train_loader, val_loader, test_loader = prepare_data(train_data, val_data, test_data)
  print("Train, validation, and test data loader size:")
  print(len(train_loader), len(val_loader), len(test_loader))

  teacher_model = load_model()
  student_model = load_model(pretrained=False,width_mult=0.25)

  print("Initial teacher Model Performance:")
  evaluate_model(teacher_model, val_loader)
  print("\nInitial Student Model Perfomance:")
  evaluate_model(student_model, val_loader)

  print("\nTraining Teacher Model:")
  teacher_model = train_model(teacher_model, train_loader, val_loader, 10)
  print("\nTeacher Model Performance after training:")
  evaluate_model(teacher_model, val_loader)

  print("\nTraining Student Model:")
  student_model = train_model(student_model, train_loader, val_loader, 50)
  print("\nStudent Model Performance after training:")
  evaluate_model(student_model, val_loader)

  print("\nDistillation:")
  student_model_distilled = load_model(pretrained=False, width_mult=0.25)
  distilled_model = model_distillation(teacher_model, student_model_distilled, train_loader, val_loader, 50)
  print("\nStudent Model Performance after distillation:")
  evaluate_model(distilled_model, val_loader)

100%|██████████| 170M/170M [00:03<00:00, 48.7MB/s]


Extracting /content/data/cifar-10-python.tar.gz to /content/data
Files already downloaded and verified
Train, validation, and test data size:
1536 512 512
Train, validation, and test data loader size:
96 32 32


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 62.1MB/s]


Initial teacher Model Performance:


100%|██████████| 32/32 [00:04<00:00,  7.39it/s]


...

Validation Accuracy:- Teacher: 80.47%; Student: 41.02%
*** Epoch: 50 ***


Loss: 2.01111: 100%|██████████| 96/96 [00:06<00:00, 15.97it/s]


Validation Accuracy:- Teacher: 80.47%; Student: 40.04%

Student Model Performance after distillation:


100%|██████████| 32/32 [00:00<00:00, 33.70it/s]

Confusion Matrix:
[[32  3  4  1  1  2  0  1  4  4]
 [ 1 35  2  0  0  0  3  0  2 14]
 [ 6  0 16  9  7  8  5  2  2  1]
 [ 1  0  9 10  5 17  7  5  1  0]
 [ 4  0  7  0 17  1  6  6  3  4]
 [ 1  0  5  8  4 16  2  3  1  3]
 [ 0  2  8  3  5  2 17  2  0  1]
 [ 0  0  6  2 15  4  3 23  0  2]
 [13  1  1  0  0  2  0  0 29  5]
 [ 5 14  2  0  1  0  4  4  3 22]]
Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.62      0.56        52
           1       0.64      0.61      0.62        57
           2       0.27      0.29      0.28        56
           3       0.30      0.18      0.23        55
           4       0.31      0.35      0.33        48
           5       0.31      0.37      0.34        43
           6       0.36      0.42      0.39        40
           7       0.50      0.42      0.46        55
           8       0.64      0.57      0.60        51
           9       0.39      0.40      0.40        55

    accuracy                    

In [ ]:
distilled_model_2 = model_distillation(teacher_model, distilled_model, train_loader, val_loader, 50)
print("\nStudent Model Performance before 2nd distillation loop:")
evaluate_model(distilled_model, val_loader)

In [11]:
print("\nStudent Model Performance after 2nd distillation loop:")
evaluate_model(distilled_model_2, val_loader)


Student Model Performance after 2nd distillation loop:


100%|██████████| 32/32 [00:00<00:00, 32.99it/s]

Confusion Matrix:
[[25  6 12  1  0  1  0  0  5  2]
 [ 1 43  2  2  1  1  3  0  2  2]
 [ 4  1 22  7  4  7  7  2  1  1]
 [ 1  1 10 16  8 10  7  1  0  1]
 [ 2  0  8  4 16  7  1  6  3  1]
 [ 0  0  6  5  3 22  6  0  1  0]
 [ 0  1  5  2  2  3 24  2  1  0]
 [ 0  0  6  3 14  6  2 20  0  4]
 [11  2  1  5  0  0  1  0 30  1]
 [ 4 20  5  0  0  1  6  0  3 16]]
Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.48      0.50        52
           1       0.58      0.75      0.66        57
           2       0.29      0.39      0.33        56
           3       0.36      0.29      0.32        55
           4       0.33      0.33      0.33        48
           5       0.38      0.51      0.44        43
           6       0.42      0.60      0.49        40
           7       0.65      0.36      0.47        55
           8       0.65      0.59      0.62        51
           9       0.57      0.29      0.39        55

    accuracy                    